In [1]:
# =====================================================================
# Processo de Automatización NOKIA 3G
# =====================================================================

import os, glob, io, re
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, Border, PatternFill
from openpyxl.utils import get_column_letter
import numpy as np
from datetime import date
import unicodedata


# Ruta base (ajústala si cambia)
BASE_DIR = Path(r"C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Nokia\XML_Output\Enero")

HEADERS = "AT&T_Site_Name", "Site Name", "RNCID", "Site ID", "VERSION", "DISTNAME", "MOID", "WCELMCC", "WCELMNC", "angle", "name", "AdminCellState", "CId", "CellAdditionalInfo", "CellBarred", "CellRange", "CellType", "HSDPA64UsersEnabled", "LAC", "MEHMaxHSUPAUsers", "MaxDLPowerCapability", "PriScrCode", "PtxPrimaryCPICH", "RAC", "SAC", "SACB", "SectorID", "Tcell", "UARFCN", "URAId", "WCelState", "cellLevel", "LAT", "LON"

HEADER_WCEL = ["VERSION", "DISTNAME", "MOID", "WCELMCC", "WCELMNC", "angle", "name", "AdminCellState", "CId", "CellAdditionalInfo", "CellBarred", "CellRange", "CellType", "HSDPA64UsersEnabled", "LAC", "MEHMaxHSUPAUsers", "MaxDLPowerCapability", "PriScrCode", "PtxPrimaryCPICH", "RAC", "SAC", "SACB", "SectorID", "Tcell", "UARFCN", "URAId", "WCelState", "cellLevel"]

HEADER_WBTS = ["DISTNAME", "BTSAdditionalInfo"]


In [2]:
def read_csv_files(filename: str, header):
    filepath = str(BASE_DIR / f"{filename}")
    # Leer sólo las columnas necesarias del csv
    df = pd.read_csv(filepath, usecols=header, low_memory=False)
#    print(wcel_df.head())

    return df

In [3]:
#BASE_DIR = Path(r"C:\Users\EAlor\OneDrive - ACS Solutions\Documents\AT&T\LST Cell Ran\Nokia New\5G\D")
wcel_df = read_csv_files("WCEL.csv", HEADER_WCEL)
wbts_df = read_csv_files("WBTS.csv", HEADER_WBTS)

#print(wcel_df.head(10))
#print(wbts_df.head(10))

print("Shape WCEL original:", wcel_df.shape, "\nShape WBTS original:", wbts_df.shape)

try:
    display(wcel_df.head(10))
    display(wbts_df.head(10))
except NameError:
    # Por si no estás en notebook
    print(wcel_df.head(10).to_string(index=False))
    print(wbts_df.head(10).to_string(index=False))

Shape WCEL original: (5839, 28) 
Shape WBTS original: (731, 2)


,VERSION,DISTNAME,MOID,WCELMCC,WCELMNC,angle,cellLevel,name,AdminCellState,CId,...,PriScrCode,PtxPrimaryCPICH,RAC,SAC,SACB,SectorID,Tcell,UARFCN,URAId,WCelState
0,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24501,80942113,334,90,5.0,Macro Cell,NMUGUALEO0389_1_B,Unlocked,24501,...,433,32.0,77,24501,24501,1,0 chips,1037,77,0
1,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24502,80942112,334,90,6.0,Macro Cell,NMUGUALEO0389_2_B,Unlocked,24502,...,376,30.0,77,24502,24502,2,256 chips,1037,77,0
2,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24503,80942111,334,90,5.0,Macro Cell,NMUGUALEO0389_3_B,Unlocked,24503,...,49,30.0,77,24503,24503,3,512 chips,1037,77,0
3,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24504,80942110,334,90,3.0,Macro Cell,NMUGUALEO0389_1_H,Unlocked,24504,...,433,31.0,77,24504,24504,1,0 chips,687,77,0
4,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24505,80942109,334,90,3.0,Macro Cell,NMUGUALEO0389_2_H,Unlocked,24505,...,376,31.0,77,24505,24505,2,256 chips,687,77,0
5,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24506,80942108,334,90,3.0,Macro Cell,NMUGUALEO0389_3_H,Unlocked,24506,...,49,31.0,77,24506,24506,3,512 chips,687,77,0
6,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-301,76604527,334,90,6.0,Macro Cell,NMUAGUAGU0016_1_B,Unlocked,301,...,258,33.0,75,301,301,1,0 chips,1037,75,0
7,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-302,76604526,334,90,5.0,Macro Cell,NMUAGUAGU0016_2_B,Unlocked,302,...,254,33.0,75,302,302,2,256 chips,1037,75,0
8,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-303,76604522,334,90,6.0,Macro Cell,NMUAGUAGU0016_3_B,Unlocked,303,...,130,33.0,75,303,303,3,512 chips,1037,75,0
9,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-304,76604521,334,90,4.0,Macro Cell,NMUAGUAGU0016_1_H,Unlocked,304,...,258,30.0,75,304,304,1,0 chips,687,75,0


,DISTNAME,BTSAdditionalInfo
0,PLMN-PLMN/RNC-360/WBTS-1145,UGUALEO0389
1,PLMN-PLMN/RNC-360/WBTS-1270,UAGUAGU0016
2,PLMN-PLMN/RNC-360/WBTS-1491,UGUALEO0405
3,PLMN-PLMN/RNC-360/WBTS-1612,UAGUAGU1616
4,PLMN-PLMN/RNC-360/WBTS-1613,UAGUSFR1671
5,PLMN-PLMN/RNC-360/WBTS-1616,UAGUAGU3822
6,PLMN-PLMN/RNC-360/WBTS-1644,UAGUAGU0043
7,PLMN-PLMN/RNC-360/WBTS-1645,UAGUAGU0044
8,PLMN-PLMN/RNC-360/WBTS-1646,UAGUAGU1647
9,PLMN-PLMN/RNC-360/WBTS-1647,UAGUAGU1656


In [4]:
# =====================================================================
# 1) Insertar columna AT&TSite_Name
# =====================================================================

# Copia para no tocar los originales
wcel_df_mod = wcel_df.copy()
#wbts_df_mod = wbts_df.copy()

print("Shape original:", wcel_df_mod.shape)

wcel_df_mod.insert(0, 'AT&T_Site_Name', wcel_df_mod['name'].str[3:13])

print("Shape nuevo:", wcel_df_mod.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(wcel_df_mod.head(10))
except NameError:
    # Por si no estás en notebook
    print(wcel_df_mod.head(10).to_string(index=False))

Shape original: (5839, 28)
Shape nuevo: (5839, 29)


,AT&T_Site_Name,VERSION,DISTNAME,MOID,WCELMCC,WCELMNC,angle,cellLevel,name,AdminCellState,CId,CellAdditionalInfo,CellBarred,CellRange,CellType,HSDPA64UsersEnabled,LAC,MEHMaxHSUPAUsers,MaxDLPowerCapability,PriScrCode,PtxPrimaryCPICH,RAC,SAC,SACB,SectorID,Tcell,UARFCN,URAId,WCelState
0,GUALEO0389,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24501,80942113,334,90,5.0,Macro Cell,NMUGUALEO0389_1_B,Unlocked,24501,NMUGUALEO0389_1_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,32.0,77,24501,24501,1,0 chips,1037,77,0
1,GUALEO0389,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24502,80942112,334,90,6.0,Macro Cell,NMUGUALEO0389_2_B,Unlocked,24502,NMUGUALEO0389_2_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,30.0,77,24502,24502,2,256 chips,1037,77,0
2,GUALEO0389,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24503,80942111,334,90,5.0,Macro Cell,NMUGUALEO0389_3_B,Unlocked,24503,NMUGUALEO0389_3_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,49,30.0,77,24503,24503,3,512 chips,1037,77,0
3,GUALEO0389,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24504,80942110,334,90,3.0,Macro Cell,NMUGUALEO0389_1_H,Unlocked,24504,NMUGUALEO0389_1_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,31.0,77,24504,24504,1,0 chips,687,77,0
4,GUALEO0389,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24505,80942109,334,90,3.0,Macro Cell,NMUGUALEO0389_2_H,Unlocked,24505,NMUGUALEO0389_2_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,31.0,77,24505,24505,2,256 chips,687,77,0
5,GUALEO0389,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24506,80942108,334,90,3.0,Macro Cell,NMUGUALEO0389_3_H,Unlocked,24506,NMUGUALEO0389_3_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,49,31.0,77,24506,24506,3,512 chips,687,77,0
6,AGUAGU0016,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-301,76604527,334,90,6.0,Macro Cell,NMUAGUAGU0016_1_B,Unlocked,301,NMUAGUAGU0016_1_B,Not barred,10000,Not_defined,Disabled,40755,10.0,46.0,258,33.0,75,301,301,1,0 chips,1037,75,0
7,AGUAGU0016,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-302,76604526,334,90,5.0,Macro Cell,NMUAGUAGU0016_2_B,Unlocked,302,NMUAGUAGU0016_2_B,Not barred,10000,Not_defined,Disabled,40755,10.0,46.0,254,33.0,75,302,302,2,256 chips,1037,75,0
8,AGUAGU0016,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-303,76604522,334,90,6.0,Macro Cell,NMUAGUAGU0016_3_B,Unlocked,303,NMUAGUAGU0016_3_B,Not barred,10000,Not_defined,Disabled,40755,10.0,46.0,130,33.0,75,303,303,3,512 chips,1037,75,0
9,AGUAGU0016,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-304,76604521,334,90,4.0,Macro Cell,NMUAGUAGU0016_1_H,Unlocked,304,NMUAGUAGU0016_1_H,Not barred,10000,Not_defined,Disabled,40755,10.0,46.0,258,30.0,75,304,304,1,0 chips,687,75,0


In [5]:
# =====================================================================
# 2) Site_Name/RNCID/Site ID
# =====================================================================

print("Shape original:", wcel_df_mod.shape)

# Inserta columna RNCID
wcel_df_mod.insert(1, 'RNCID', wcel_df_mod['DISTNAME'].str.extract(r'RNC-(.{3})', expand=False))

# Inserta columna Site ID con la informacion encontrada despues de "RNC-xxx/" hasta el siguiente "/"
wcel_df_mod.insert(2, 'Site ID', wcel_df_mod['DISTNAME'].str.extract(r'RNC-\d+/([^/]+)', expand=False))


# Inserta columna Site ID
# Crear una clave en df con la concatenación compleja
clave ='PLMN-PLMN/RNC-' + wcel_df_mod['RNCID'].astype(str) + '/' + wcel_df_mod['Site ID'].astype(str)

# Crear un diccionario desde wbts_df: DISTNAME -> BTSAdditionalInfo
mapa = dict(zip(wbts_df['DISTNAME'], wbts_df['BTSAdditionalInfo']))

# Insertar la nueva columna Site Name en posición 1 con el valor mapeado
wcel_df_mod.insert(1, 'Site Name', clave.map(mapa))

'''
wcel_df_mod.insert(1, 'Site Name', wbts_df['name'].str[3:13])
'''
print("Shape nuevo:", wcel_df_mod.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(wcel_df_mod.head(10))
except NameError:
    # Por si no estás en notebook
    print(wcel_df_mod.head(10).to_string(index=False))

Shape original: (5839, 29)
Shape nuevo: (5839, 32)


,AT&T_Site_Name,Site Name,RNCID,Site ID,VERSION,DISTNAME,MOID,WCELMCC,WCELMNC,angle,cellLevel,name,AdminCellState,CId,CellAdditionalInfo,CellBarred,CellRange,CellType,HSDPA64UsersEnabled,LAC,MEHMaxHSUPAUsers,MaxDLPowerCapability,PriScrCode,PtxPrimaryCPICH,RAC,SAC,SACB,SectorID,Tcell,UARFCN,URAId,WCelState
0,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24501,80942113,334,90,5.0,Macro Cell,NMUGUALEO0389_1_B,Unlocked,24501,NMUGUALEO0389_1_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,32.0,77,24501,24501,1,0 chips,1037,77,0
1,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24502,80942112,334,90,6.0,Macro Cell,NMUGUALEO0389_2_B,Unlocked,24502,NMUGUALEO0389_2_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,30.0,77,24502,24502,2,256 chips,1037,77,0
2,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24503,80942111,334,90,5.0,Macro Cell,NMUGUALEO0389_3_B,Unlocked,24503,NMUGUALEO0389_3_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,49,30.0,77,24503,24503,3,512 chips,1037,77,0
3,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24504,80942110,334,90,3.0,Macro Cell,NMUGUALEO0389_1_H,Unlocked,24504,NMUGUALEO0389_1_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,31.0,77,24504,24504,1,0 chips,687,77,0
4,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24505,80942109,334,90,3.0,Macro Cell,NMUGUALEO0389_2_H,Unlocked,24505,NMUGUALEO0389_2_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,31.0,77,24505,24505,2,256 chips,687,77,0
5,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24506,80942108,334,90,3.0,Macro Cell,NMUGUALEO0389_3_H,Unlocked,24506,NMUGUALEO0389_3_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,49,31.0,77,24506,24506,3,512 chips,687,77,0
6,AGUAGU0016,UAGUAGU0016,360,WBTS-1270,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-301,76604527,334,90,6.0,Macro Cell,NMUAGUAGU0016_1_B,Unlocked,301,NMUAGUAGU0016_1_B,Not barred,10000,Not_defined,Disabled,40755,10.0,46.0,258,33.0,75,301,301,1,0 chips,1037,75,0
7,AGUAGU0016,UAGUAGU0016,360,WBTS-1270,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-302,76604526,334,90,5.0,Macro Cell,NMUAGUAGU0016_2_B,Unlocked,302,NMUAGUAGU0016_2_B,Not barred,10000,Not_defined,Disabled,40755,10.0,46.0,254,33.0,75,302,302,2,256 chips,1037,75,0
8,AGUAGU0016,UAGUAGU0016,360,WBTS-1270,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-303,76604522,334,90,6.0,Macro Cell,NMUAGUAGU0016_3_B,Unlocked,303,NMUAGUAGU0016_3_B,Not barred,10000,Not_defined,Disabled,40755,10.0,46.0,130,33.0,75,303,303,3,512 chips,1037,75,0
9,AGUAGU0016,UAGUAGU0016,360,WBTS-1270,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1270/WCEL-304,76604521,334,90,4.0,Macro Cell,NMUAGUAGU0016_1_H,Unlocked,304,NMUAGUAGU0016_1_H,Not barred,10000,Not_defined,Disabled,40755,10.0,46.0,258,30.0,75,304,304,1,0 chips,687,75,0


In [6]:
# =====================================================================
# 5) LAT/LON desde All_Nokia_3G_{YYYYMM} (mes anterior)
# =====================================================================
wcel_df_mergelatlon = wcel_df_mod.copy()

today = date.today()
prev_year = today.year if today.month > 1 else today.year - 1
prev_month = today.month - 1 or 12
yyyymm = f"{prev_year}{prev_month:02d}"
print(yyyymm)

# an_path = BASE_DIR / f"All_Nokia_3G_{yyyymm}.xlsx"
an_path = BASE_DIR / f"All_Nokia_3G_20260121.xlsx"
an_df = pd.read_excel(an_path, usecols=["AT&T_Site_Name", "LAT", "LON"])
an_df["AT&T_Site_Name"] = an_df["AT&T_Site_Name"].astype(str).str.strip()
an_df = an_df.drop_duplicates(subset=["AT&T_Site_Name"], keep="first")
display(an_df.head(5))

wcel_df_mergelatlon["AT&T_Site_Name"] = wcel_df_mergelatlon["AT&T_Site_Name"].astype(str).str.strip()

merged_df = wcel_df_mergelatlon.merge(
    an_df,
    on="AT&T_Site_Name",
    how="left",
    suffixes=("", "_an")
)


print("Shape nuevo:", merged_df.shape)

# Vista de verificación (muestra solo unas filas)
pd.set_option("display.max_columns", None)  # opcional
try:
    display(merged_df.head(5))
except NameError:
    print(merged_df.head(5).to_string(index=False))


202512


,AT&T_Site_Name,LAT,LON
0,AGUAGU1616,21.864416,-102.258916
6,AGUSFR1671,21.954264,-102.253075
12,AGUAGU3822,21.886288,-102.294046
18,AGUAGU0043,21.907806,-102.237056
24,AGUAGU0044,21.927450,-102.338100


Shape nuevo: (5839, 34)


,AT&T_Site_Name,Site Name,RNCID,Site ID,VERSION,DISTNAME,MOID,WCELMCC,WCELMNC,angle,cellLevel,name,AdminCellState,CId,CellAdditionalInfo,CellBarred,CellRange,CellType,HSDPA64UsersEnabled,LAC,MEHMaxHSUPAUsers,MaxDLPowerCapability,PriScrCode,PtxPrimaryCPICH,RAC,SAC,SACB,SectorID,Tcell,UARFCN,URAId,WCelState,LAT,LON
0,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24501,80942113,334,90,5.0,Macro Cell,NMUGUALEO0389_1_B,Unlocked,24501,NMUGUALEO0389_1_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,32.0,77,24501,24501,1,0 chips,1037,77,0,21.167794,-101.685682
1,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24502,80942112,334,90,6.0,Macro Cell,NMUGUALEO0389_2_B,Unlocked,24502,NMUGUALEO0389_2_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,30.0,77,24502,24502,2,256 chips,1037,77,0,21.167794,-101.685682
2,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24503,80942111,334,90,5.0,Macro Cell,NMUGUALEO0389_3_B,Unlocked,24503,NMUGUALEO0389_3_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,49,30.0,77,24503,24503,3,512 chips,1037,77,0,21.167794,-101.685682
3,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24504,80942110,334,90,3.0,Macro Cell,NMUGUALEO0389_1_H,Unlocked,24504,NMUGUALEO0389_1_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,31.0,77,24504,24504,1,0 chips,687,77,0,21.167794,-101.685682
4,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24505,80942109,334,90,3.0,Macro Cell,NMUGUALEO0389_2_H,Unlocked,24505,NMUGUALEO0389_2_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,31.0,77,24505,24505,2,256 chips,687,77,0,21.167794,-101.685682


In [8]:
# =====================================================================
# 6) LAT/LON desde EPT (mas reciente)
# =====================================================================

if merged_df[["LAT", "LON"]].isna().any(axis=1).any():
    # Ejecuta el proceso si hay al menos un NaN en LAT o LON

    nan_count = merged_df[["LAT", "LON"]].isna().sum().sum()
    print("Se necesita buscar LAT/LON restantes en EPT")

    ruta_ept = Path(BASE_DIR)

    # Prefijo del archivo
    prefijo_ept = "EPT_ATT_UMTS_LTE_"

    # Busca archivo que empiece con el prefijo
    archivo = glob.glob(os.path.join(ruta_ept, f"{prefijo_ept}*.xlsx"))

    # Verifica si se encontró archivo
    if archivo:
        archivo_encontrado = archivo[0]
        nombre_archivo = os.path.basename(archivo_encontrado)

        # Lista de hojas a leer
        hojas_fijas = [
            "EPT_3G_LTE_OUTDOOR",
            "PLAN_OUTDOOR",
            "EPT_3G_LTE_INDOOR",
            "PLAN_INDOOR",
            "Eventos_Especiales"
        ]

        # Detecta automáticamente las hojas que contienen "Nokia" (para este vendor en particular)
        todas_las_hojas = pd.ExcelFile(archivo_encontrado, engine="openpyxl").sheet_names
        hojas_vendor = [h for h in todas_las_hojas if "nokia" in h.lower()]
        # Combina ambas listas (sin duplicar)
        hojas = list(dict.fromkeys(hojas_fijas + hojas_vendor))
        print(hojas)

        # Lee todas las hojas y agrega el nombre de la hoja en columna

        dfs = [
            pd.read_excel(archivo_encontrado, sheet_name=hoja, usecols=["AT&T_Site_Name", "Latitud", "Longitud"],
                          engine="openpyxl")
            .assign(Hoja=hoja, Origen=nombre_archivo)
            for hoja in hojas
        ]

        # Concatena todo en un solo DataFrame
        df_EPT_inicial = pd.concat(dfs, ignore_index=True).drop_duplicates(subset=["AT&T_Site_Name"])

        # Unir con df principal (solo para los NaN)
        merged_df = merged_df.merge(df_EPT_inicial, on="AT&T_Site_Name", how="left", suffixes=("", "_extra"))
        merged_df["LAT"] = merged_df["LAT"].fillna(merged_df["Latitud"])
        merged_df["LON"] = merged_df["LON"].fillna(merged_df["Longitud"])
        merged_df = merged_df.drop(columns=["Latitud", "Longitud"])

        if merged_df[["LAT", "LON"]].isna().any(axis=1).any():
            nan_count = merged_df[["LAT", "LON"]].isna().sum().sum()
            print(nan_count, " LAT/LON no encontrados. Se producirá archivo excel con estos faltantes.")

        # Vista de verificación (muestra solo unas filas)
        pd.set_option("display.max_columns", None)  # opcional
        try:
            display(merged_df.head(5))
        except NameError:
            # Por si no estás en notebook
            print(merged_df.head(5).to_string(index=False))
    else:
        print("⚠️ No se encontró archivo EPT")
else:
    print("LAT y LON encontrados en su totalidad en archivo anterior. No se necesita EPT.")

# Eliminar columnas auxiliares si existen
merged_df = merged_df.drop(columns=[c for c in ["Hoja", "Origen"] if c in merged_df.columns])


Se necesita buscar LAT/LON restantes en EPT
['EPT_3G_LTE_OUTDOOR', 'PLAN_OUTDOOR', 'EPT_3G_LTE_INDOOR', 'PLAN_INDOOR', 'Eventos_Especiales', 'Overlay Nokia', 'Nokia_DSS', 'R&R Nokia']


C:\Users\SCaracoza\AppData\Local\Temp\ipykernel_13432\3751662988.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_EPT_inicial = pd.concat(dfs, ignore_index=True).drop_duplicates(subset=["AT&T_Site_Name"])


,AT&T_Site_Name,Site Name,RNCID,Site ID,VERSION,DISTNAME,MOID,WCELMCC,WCELMNC,angle,cellLevel,name,AdminCellState,CId,CellAdditionalInfo,CellBarred,CellRange,CellType,HSDPA64UsersEnabled,LAC,MEHMaxHSUPAUsers,MaxDLPowerCapability,PriScrCode,PtxPrimaryCPICH,RAC,SAC,SACB,SectorID,Tcell,UARFCN,URAId,WCelState,LAT,LON,Hoja,Origen
0,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24501,80942113,334,90,5.0,Macro Cell,NMUGUALEO0389_1_B,Unlocked,24501,NMUGUALEO0389_1_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,32.0,77,24501,24501,1,0 chips,1037,77,0,21.167794,-101.685682,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-01-27.xlsx
1,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24502,80942112,334,90,6.0,Macro Cell,NMUGUALEO0389_2_B,Unlocked,24502,NMUGUALEO0389_2_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,30.0,77,24502,24502,2,256 chips,1037,77,0,21.167794,-101.685682,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-01-27.xlsx
2,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24503,80942111,334,90,5.0,Macro Cell,NMUGUALEO0389_3_B,Unlocked,24503,NMUGUALEO0389_3_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,49,30.0,77,24503,24503,3,512 chips,1037,77,0,21.167794,-101.685682,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-01-27.xlsx
3,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24504,80942110,334,90,3.0,Macro Cell,NMUGUALEO0389_1_H,Unlocked,24504,NMUGUALEO0389_1_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,31.0,77,24504,24504,1,0 chips,687,77,0,21.167794,-101.685682,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-01-27.xlsx
4,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24505,80942109,334,90,3.0,Macro Cell,NMUGUALEO0389_2_H,Unlocked,24505,NMUGUALEO0389_2_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,31.0,77,24505,24505,2,256 chips,687,77,0,21.167794,-101.685682,EPT_3G_LTE_OUTDOOR,EPT_ATT_UMTS_LTE_2026-01-27.xlsx


In [9]:
# =====================================================================
# 7) Archivo final en excel
# =====================================================================
# === 0) Config y fecha actual ===
# Eliminar columnas auxiliares si existen
merged_df = merged_df.drop(columns=[c for c in ["Hoja", "Origen"] if c in merged_df.columns])

today   = date.today()
yyyymm  = f"{today.year}{today.month:02d}{today.day:02d}"

final_excel = BASE_DIR / f"All_Nokia_3G_{yyyymm}.xlsx"
tmp_excel   = BASE_DIR / f"~tmp_All_Nokia_3G_{yyyymm}.xlsx"

# Usa tu DataFrame final en memoria
df_out = merged_df.copy()  # o df_sorted si ya lo traes ordenado

# 2) Garantizar columnas y orden
for col in HEADERS:
    if col not in df_out.columns:
        df_out[col] = None
# evita duplicados conservando la primera aparición
df_out = df_out.loc[:, ~pd.Index(df_out.columns).duplicated()]
# alinear exactamente a HEADERS
aligned = pd.DataFrame(index=df_out.index, columns=HEADERS, dtype=object)
for c in HEADERS:
    if c in df_out.columns:
        aligned[c] = df_out[c]
    else:
        aligned[c] = None
df_out = aligned

pd.set_option("display.max_columns", None)  # opcional
try:
    display(df_out.head(5))
except NameError:
    # Por si no estás en notebook
    print(df_out.head(5).to_string(index=False))

# === 2) Guardar sin formato, sin 'nan' ===
df_out.to_excel(final_excel, index=False, na_rep="")

# 2) Reabrir el MISMO archivo y aplicar formato
wb = load_workbook(final_excel)
ws = wb.active

ws.freeze_panes = "A2"
for col_idx, header in enumerate(HEADERS, start=1):
    cell = ws.cell(row=1, column=col_idx)
    cell.value = header
    cell.font = Font(name="Aptos Narrow", size=11)
    cell.alignment = Alignment(textRotation=90, horizontal="center", vertical="center", wrap_text=True)
    cell.border = Border()

# Agregar filtro automático en el header
ws.auto_filter.ref = ws.dimensions  # aplica el filtro a todo el rango con datos

wb.save(final_excel)
wb.close()

print(f"✅ Archivo final → {final_excel}")

,AT&T_Site_Name,Site Name,RNCID,Site ID,VERSION,DISTNAME,MOID,WCELMCC,WCELMNC,angle,name,AdminCellState,CId,CellAdditionalInfo,CellBarred,CellRange,CellType,HSDPA64UsersEnabled,LAC,MEHMaxHSUPAUsers,MaxDLPowerCapability,PriScrCode,PtxPrimaryCPICH,RAC,SAC,SACB,SectorID,Tcell,UARFCN,URAId,WCelState,cellLevel,LAT,LON
0,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24501,80942113,334,90,5.0,NMUGUALEO0389_1_B,Unlocked,24501,NMUGUALEO0389_1_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,32.0,77,24501,24501,1,0 chips,1037,77,0,Macro Cell,21.167794,-101.685682
1,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24502,80942112,334,90,6.0,NMUGUALEO0389_2_B,Unlocked,24502,NMUGUALEO0389_2_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,30.0,77,24502,24502,2,256 chips,1037,77,0,Macro Cell,21.167794,-101.685682
2,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24503,80942111,334,90,5.0,NMUGUALEO0389_3_B,Unlocked,24503,NMUGUALEO0389_3_B,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,49,30.0,77,24503,24503,3,512 chips,1037,77,0,Macro Cell,21.167794,-101.685682
3,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24504,80942110,334,90,3.0,NMUGUALEO0389_1_H,Unlocked,24504,NMUGUALEO0389_1_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,433,31.0,77,24504,24504,1,0 chips,687,77,0,Macro Cell,21.167794,-101.685682
4,GUALEO0389,UGUALEO0389,360,WBTS-1145,ASRNCFP20C,PLMN-PLMN/RNC-360/WBTS-1145/WCEL-24505,80942109,334,90,3.0,NMUGUALEO0389_2_H,Unlocked,24505,NMUGUALEO0389_2_H,Not barred,10000,Not_defined,Disabled,40717,10.0,46.0,376,31.0,77,24505,24505,2,256 chips,687,77,0,Macro Cell,21.167794,-101.685682


✅ Archivo final → C:\Users\SCaracoza\Documents\AT&T\LST Cell Ran\Nokia\XML_Output\Enero\All_Nokia_3G_20260129.xlsx
